## 1. Import dependencies

In [1]:
#pip install stable-baselines3[extra]

In [2]:
import numpy as np

In [3]:
# pip install --upgrade numpy

In [4]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [5]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder

import time

In [8]:
!pip install pyvirtualdisplay opencv-python


In [11]:
import cv2
from IPython.display import Video

## 2. Load Environment

In [6]:
environment_name = 'CartPole-v1'
env = gym.make(environment_name, render_mode='human')

In [7]:
environment_name

'CartPole-v1'

In [24]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter('rl_simulation.mp4', fourcc, 24.0, (600, 400))

for episode in range(1,episodes +1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render() #Allows us to view the grpahical representaation of environment
        time.sleep(0.1)
        # Render the environment
        frame = env.render()
        # Check if the frame is not empty
        if frame is not None and frame.shape[0] > 0 and frame.shape[1] > 0:
            frame = cv2.resize(frame, (600, 400))

        # Write the frame to the video
        video_writer.write(frame)

        action = env.action_space.sample()
        n_state, reward, done,truncated, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

# Release the video writer
video_writer.release()

# Display the video
Video("rl_simulation.mp4")
#env.close()


Episode:1 Score:20.0
Episode:2 Score:22.0
Episode:3 Score:14.0
Episode:4 Score:19.0
Episode:5 Score:57.0


In [25]:
env.reset()

(array([ 0.01080834, -0.0189237 , -0.04048218, -0.03012205], dtype=float32),
 {})

In [26]:
env.reset()

(array([-0.03137867,  0.04174085, -0.03695783,  0.00263751], dtype=float32),
 {})

## 3. Understanding Environment

In [27]:
env.action_space

Discrete(2)

In [28]:
env.action_space.sample()

1

In [29]:
# Observation space
env.observation_space.sample()

array([ 2.6895094e+00,  2.3753533e+38, -1.6085929e-01, -1.2720713e+38],
      dtype=float32)

In [30]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [31]:
env.close()

### Train a RL Model

In [32]:
# We are looking for Model Free based Learning since it can be used to 

In [33]:
# Making the directories
log_path = os.path.join('Training','Logs' )

In [34]:
log_path

'Training\\Logs'

In [35]:
env = gym.make(environment_name, render_mode='human')
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [58]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [36]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_24
-----------------------------
| time/              |      |
|    fps             | 36   |
|    iterations      | 1    |
|    time_elapsed    | 56   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 39          |
|    iterations           | 2           |
|    time_elapsed         | 103         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008762405 |
|    clip_fraction        | 0.108       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00736     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.59        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.017      |
|    value_loss           | 50.6        |
-----------------------------------------
--

## 4. Save and Reload Model

In [37]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [38]:
model.save(PPO_Path)

In [62]:
del model

In [63]:
PPO_Path

'Training\\Saved Models\\PPO_Model_Cartpole'

In [64]:
model = PPO.load(PPO_Path, env=env)

In [65]:
model.learn(total_timesteps=1000)

Logging to Training\Logs\PPO_23
-----------------------------
| time/              |      |
|    fps             | 48   |
|    iterations      | 1    |
|    time_elapsed    | 42   |
|    total_timesteps | 2048 |
-----------------------------


## 5. Evaluation Metrics

In [39]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\ASUS\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(500.0, 0.0)

## 6.Test Model

In [40]:
action, _ = model.predict(obs)

NameError: name 'obs' is not defined

In [41]:

episodes = 5
for episode in range(1,episodes +1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render() #Allows us to view the grpahical representaation of environment
        action, _ = model.predict(obs) #Now using model.predict
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))


Episode:1 Score:[411.]
Episode:2 Score:[500.]
Episode:3 Score:[345.]
Episode:4 Score:[500.]
Episode:5 Score:[500.]


In [ ]:
env.close()